# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps
import json
from api_keys import g_key

# Import API key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
city_data = "cities.csv"
city_data_df = pd.read_csv(city_data, encoding="ISO-8859-1")

city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,Aksha,50,RU,1584162325,60,50.28,113.29,-0.17,7.33
1,Kamenka,100,RU,1584162269,93,51.32,42.77,1.11,1.79
2,Rosetown,100,CA,1584162326,65,51.55,-108.00,-15.00,5.36
3,Leopold,90,AU,1584162326,67,-38.18,144.47,17.78,9.80
4,Akdepe,6,TM,1584162326,43,42.06,59.38,11.00,4.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
city_data_df = city_data_df.dropna()

locations = city_data_df[["Latitude","Longitude"]]
# Fill NaN values and convert to float
weight = city_data_df["Humidity"]
weight

0      60
1      93
2      65
3      67
4      43
       ..
565    19
566    29
567    94
568    93
569    79
Name: Humidity, Length: 565, dtype: int64

In [8]:
gmaps.configure(api_key=g_key)

#Create a list containing coordinates
# Plot Heatmap
fig = gmaps.figure(center=(32,14),  zoom_level=1)
weight
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weight, 
                                 dissipating=False, max_intensity=200,
                                 point_radius= 2)

# Add layer
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.
    
hotel_df = city_data_df[(city_data_df["Maximum Temperature"] < 27) & (city_data_df["Maximum Temperature"] > 21) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df.loc[:,"Hotel Name"] = ""

In [11]:


# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
# print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
            print("Missing field/result... skipping.")
hotel_df.head()

Missing field/result... skipping.


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed,Hotel Name
33,Cidreira,0,BR,1584162219,89,-30.18,-50.21,23.82,9.25,OYO Hotel Pacífico
59,Chui,0,UY,1584162265,88,-33.70,-53.46,21.24,4.06,Turis Firper Hotel
65,Arraial do Cabo,0,BR,1584162244,85,-22.97,-42.03,25.05,4.46,Paradiso Peró Praia Hotel
71,Kaya,0,BF,1584162335,12,13.09,-1.08,23.19,4.51,Pacific Hotel Kaya
95,San Patricio,0,MX,1584162242,85,19.22,-104.70,23.90,2.43,Las Hadas by Brisas Manzanillo


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# syntax for marker layer (locations, hover_text='', label='', info_box_content=None, display_info_box=None)

# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))